<h3> 현재 구축 상황 </h3>
폴더에서 파일 불러와서 result란 폴더를 생성 
음원 파일이 있는 폴더에 대해 mfcc 연산을 하고 그 plot을 result 폴더에 저장할 수 있다.<br>
mfcc 데이터 csv file로 저장 가능<br>
csv format의 data_description을 통해서 mfcc data를 emotion에 따라 여러 폴더에 나눠서 저장<br> 

<h3> 해야할 것 </h3>

병렬 연산에 되면 무조건 찾아봐야 함. 데이터 너무 많음.

In [1]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import matplotlib.cm as cm
import pandas as pd
import joblib # multiprocessing에 필요한 라이브러리

In [2]:
# 문자로 데이터 찾기
def find_emotion(csvfile, str): 
    return csvfile[csvfile['wav_id'].str.contains(str)].iloc[0][1]


In [3]:
# 멀티프로세싱을 위해서 mfcc 구동 및 결과 출력 과정을 함수로 만듦
# joblib의 입력값이 튜플이기 때문에 cnt, audio_list 가지고 고생할 필요 X
def mfcc_multi(y):
    
    # 경로 확장자 분리 참조 : https://jvvp.tistory.com/980
    y = ''.join(y) # y가 튜플타입이기 때문에 문자열로 변환해주어야함.
    print(y) # audio file명 나옴(확장자와 같이)
    audio_path = path + '/' + y
    file_code, ext = os.path.splitext(y)  # 확장자 분리
    dirname, basename = os.path.split(file_code)  # 경로 분리
    y, sr = librosa.load(audio_path, 16000)
        
    frame_length = 0.025
    frame_stride = 0.01
    sr = 16000

    n_fft_25 = int(round(sr * frame_length))
    hop_length_10 = int(round(sr * frame_stride))

    n_fft = n_fft_25 * 2
    win_length = n_fft_25
    hop_length = hop_length_10
    n_mels = 128  # default
    n_mfcc = 20  # default

    # mfcc에는 멜 스펙트로그램의 log값이 필요하다.
    ## 원래는 logamplitude 함수로 변환하였으나, power를 db로 변환하는 과정 자체가 로그함수를 씌우는 것이기 때문에 power_to_db로 함수가 대체됨

    # audio file -> MFCC data
    D = np.abs(librosa.stft(y, n_fft=n_fft, win_length=win_length, hop_length=hop_length))
    MS = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length,
                                            win_length=win_length)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(D), sr=sr, n_mfcc=n_mfcc)

    # numpy array csv로 추출 참조 : https://appia.tistory.com/204
    if find_emotion(pd_csv_id, basename) == 'anger':
        ext_name = path + '/res_anger/' + basename + '.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'sad':
        ext_name = path + '/res_sad/' + basename + '.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'disgust':
        ext_name = path + '/res_disg/' + basename + '.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    elif find_emotion(pd_csv_id, basename) == 'fear':
        ext_name = path + '/res_fear/' + basename + '.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")
    else:
        ext_name = path + '/res_else/' + basename + '.csv'
        np.savetxt(ext_name, mfccs, delimiter=",")



In [4]:
# 폴더 생성
## 출처 : https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


In [6]:
# 파이썬 파일 생성하듯이 실행
if __name__ == "__main__":
    path = 'C:/Users/RestJSM/librosa/data/emotion_dataset/test'

    audio_list = os.listdir(path)

    pd_path = 'C:/Users/RestJSM/librosa/data/emotion_dataset'
    pd_csv = pd.read_csv(pd_path + '/5th_year.csv', encoding = 'cp949')
    pd_csv_id = pd_csv.loc[:, ['wav_id', '상황']]
    print(pd_csv_id)

    createFolder(path + '/res_anger')
    createFolder(path + '/res_sad')
    createFolder(path + '/res_neutral')
    createFolder(path + '/res_disg')
    createFolder(path + '/res_fear')
    createFolder(path + '/res_else')


    n_jobs = 8
    verbose = 1

    jobs = [joblib.delayed(mfcc_multi)(audio) for audio in zip(audio_list)]
    out = joblib.Parallel(n_jobs=n_jobs, verbose=verbose)(jobs)
    print(out)

                         wav_id       상황
0      5ed10dbc2880d70f286121c3  disgust
1      5ecb60ef9aa8ea0eec53edb1  disgust
2      5f052858b140144dfcfef768      sad
3      5f0e72c4b140144dfcff3fa5      sad
4      5ed10ddd7e21a10eee2537ce  disgust
...                         ...      ...
10006  5f05ffd5b140144dfcff01c7     fear
10007  5ec53bf82880d70f28611de0      sad
10008  5f0b2b5ab140144dfcff2759     fear
10009  5f0b2b75b140144dfcff275a     fear
10010  5f0527b3b140144dfcfef75e     fear

[10011 rows x 2 columns]


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    8.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   19.2s


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

[Parallel(n_jobs=8)]: Done 334 out of 334 | elapsed:   32.7s finished
